In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from evaluation.analysis import Plotter
plt = Plotter("D:/Guy_Gilad/Evaluation/Eval_v2/bboxes.csv")
plt.plot_v1()

In [ ]:
data = plt._data
mask = data['bbox_area_diff'] <= 0.2
data[mask].describe()

In [ ]:
plt._data = data[mask]
plt.plot_v1()

In [ ]:
import seaborn as sns
data = plt._data
data['wrm_speed_avg'] = data['wrm_speed'].rolling(window=5, center=True).mean()
ax = sns.scatterplot(data=plt._data, x='wrm_speed_avg', y='bbox_area_diff')
ax.set_xlim(0,10)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
import cv2 as cv
import numpy as np
from tqdm.auto import tqdm
import math

from frame_reader import FrameReader


class WormExtractor:
    def __init__(
        self,
        frame_reader: FrameReader,
        bg_probes: int = 100,
        diff_thresh: int = 10,
    ):
        assert bg_probes > 0 and diff_thresh > 0

        self._frame_reader = frame_reader
        self._bg_probes = bg_probes
        self._diff_thresh = diff_thresh

        self._background = None

    def get_background(self) -> np.ndarray:
        """
        Returns the background image extracted from the frame reader frames.

        Returns:
            np.ndarray: The background array.
        """

        if self._background is None:
            self._background = self._calc_background()
        return self._background

    def _calc_background(self) -> np.ndarray:
        length = len(self._frame_reader)
        size = min(self._bg_probes, length)

        # get frames
        frame_ids = np.random.choice(length, size=size, replace=False)
        extracted_list = []
        for frame_id in tqdm(frame_ids, desc="Extracting background frames", unit="fr"):
            frame = self._frame_reader[frame_id]
            extracted_list.append(frame)

        # calculate the median along the time axis
        extracted = np.stack(extracted_list, axis=0)
        median = np.median(extracted, axis=0).astype(np.uint8, copy=False)
        return median

    def find_worm(self, frame_idx: int, bbox: tuple[float]) -> np.ndarray:
        """
        Returns a binary mask of the worm in the frame, within the bounding box.

        Args:
            frame_idx (int): The frame index.
            bbox (tuple[float]): The bounding box (x, y, w, h).

        Returns:
            np.ndarray: The binary mask of the worm.
        """

        # get mask according to the threshold value
        frame = self._frame_reader[frame_idx]
        background = self.get_background()

        x, y, w, h = math.floor(bbox[0]), math.floor(bbox[1]), math.ceil(bbox[2]), math.ceil(bbox[3])

        # extract the region of interest
        roi = frame[y : y + h, x : x + w]
        bg_roi = background[y : y + h, x : x + w]

        # make sure roi and bg are grayscale
        if len(roi.shape) == 3:
            roi = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
            bg_roi = cv.cvtColor(bg_roi, cv.COLOR_BGR2GRAY)

        diff = cv.absdiff(roi, bg_roi)
        _, mask = cv.threshold(diff, self._diff_thresh, 255, cv.THRESH_BINARY)

        # apply morphological ops to the mask
        mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((5, 5), np.uint8))
        mask = cv.dilate(mask, np.ones((11, 11), np.uint8))

        # extract contours and bbox
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        worm_binary = np.zeros_like(roi)

        if len(contours) > 0:
            largest_contour = max(contours, key=cv.contourArea)
            cv.drawContours(worm_binary, [largest_contour], 0, 1, cv.FILLED)

        return worm_binary

In [ ]:
class ErrorCalculator:
    def __init__(self, frame_reader: FrameReader, worm_extractor: WormExtractor):
        self._frame_reader = frame_reader
        self._worm_extractor = worm_extractor

    def calculate_error(self, frame_idx: int, worm_bbox: tuple[float], mic_bbox: tuple[float]) -> float:
        worm_bbox = math.floor(worm_bbox[0]), math.floor(worm_bbox[1]), math.ceil(worm_bbox[2]), math.ceil(worm_bbox[3])
        mic_bbox = math.floor(mic_bbox[0]), math.floor(mic_bbox[1]), math.ceil(mic_bbox[2]), math.ceil(mic_bbox[3])

        worm_mask = self._worm_extractor.find_worm(frame_idx, worm_bbox)

        num_pixels = np.sum(worm_mask)

        if num_pixels == 0:
            return 0

        # calculate the intersection between the worm and the mic bounding box
        x, y, w, h = (
            max(0, mic_bbox[0] - worm_bbox[0]),
            max(0, mic_bbox[1] - worm_bbox[1]),
            min(worm_bbox[0] + worm_bbox[2], mic_bbox[0] + mic_bbox[2]),
            min(worm_bbox[1] + worm_bbox[3], mic_bbox[1] + mic_bbox[3]),
        )

        worm_intersection = worm_mask[y : y + h, x : w + w]

        return np.sum(worm_intersection) / num_pixels

In [ ]:
extractor = WormExtractor(reader)

In [ ]:
bbox = (360 - 360 // 2, 2348 - 360 // 2, 25, 13)
mask = extractor.find_worm(15, (0, 0, 10000, 10000))

In [ ]:
np.sum(mask != 0)

In [ ]:
np.sum(mask == 1)

In [ ]:
colored = cv.cvtColor(mask * 255, cv.COLOR_GRAY2BGR)
cv.imshow("mask", colored)
cv.waitKey(0)

In [1]:
from evaluation.simulator import *
from evaluation.sim_controllers import *
from utils.path_utils import Files

time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.225, 0.225),
    init_position=(1300, 1200),
    frame_padding_value=(255, 255, 255),
)

log_config = LogConfig(
    root_folder="logs_v2/",
    save_mic_view=True,
    save_cam_view=True,
)

log_config.create_dirs()

"""
time_config.save_json("logs/time_config.json")
log_config.save_json("logs/log_config.json")
yolo_config.save_json("logs/yolo_config.json")
"""

time_config.save_json(log_config.root_folder+"/time_config.json")
log_config.save_json(log_config.root_folder+"/log_config.json")
yolo_config.save_json(log_config.root_folder+"/yolo_config.json")

""" time_config = TimingConfig.load_json("logs/time_config.json")
log_config = LogConfig.load_json("logs/log_config.json")
yolo_config = YoloConfig.load_json("logs/yolo_config.json") """


' time_config = TimingConfig.load_json("logs/time_config.json")\nlog_config = LogConfig.load_json("logs/log_config.json")\nyolo_config = YoloConfig.load_json("logs/yolo_config.json") '

In [ ]:
print(time_config.imaging_frame_num + time_config.moving_frame_num)

In [2]:
sim_controller = YoloController(time_config, yolo_config)
sim_controller = CsvController(time_config, "logs/bboxes.csv")
log_controller = LoggingController(sim_controller, log_config)
sim = Simulator(time_config, reader, log_controller)

In [3]:
sim.run(visualize=True)

Simulation Progress:   0%|          | 0/14 [00:00<?, ?cycle/s]

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")